In [1]:
install.packages("lavaan", dependencies = TRUE)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [8]:
library(lavaan)

This is lavaan 0.6-1
lavaan is BETA software! Please report any bugs.


In [2]:
dat<-read.csv("https://github.com/thousandoaks/SEM/blob/master/SEM_06_SEM_versus_Multiple_Regression_data.csv?raw=true")


In [4]:
head(dat)

distance,elev,abiotic,age,hetero,firesev,cover,rich
53.40900,1225,60.67103,40,0.757065,3.50,1.0387974,51
37.03745,60,40.94291,25,0.491340,4.05,0.4775924,31
53.69565,200,50.98805,15,0.844485,2.60,0.9489357,71
53.69565,200,61.15633,15,0.690847,2.90,1.1949002,64
51.95985,970,46.66807,23,0.545628,4.30,1.2981890,68
51.95985,970,39.82357,24,0.652895,4.00,1.1734866,34


### Let's start with model 1

In [13]:
mod.1 <- 'rich ~ abiotic + hetero + distance + firesev + 0*age
    abiotic ~ distance
    hetero ~ distance
    age ~ distance
    firesev ~ age'

In [14]:
mod.1.fit <- sem(mod.1, data=dat)

Warning message in lav_data_full(data = data, group = group, cluster = cluster, :
“lavaan WARNING: some observed variances are (at least) a factor 1000 times larger than others; use varTable(fit) to investigate”

In [32]:
varTable(mod.1.fit)

name,idx,nobs,type,exo,user,mean,var,nlev,lnam
rich,8,90,numeric,0,0,49.2333333,228.1808989,0,
abiotic,3,90,numeric,0,0,49.2390250,58.9687166,0,
hetero,5,90,numeric,0,0,0.6833189,0.0131832,0,
age,4,90,numeric,0,0,25.5666667,157.9112360,0,
firesev,6,90,numeric,0,0,4.5650000,2.7302500,0,
distance,1,90,numeric,1,0,49.2345833,77.9597236,0,


In [29]:
## Recode vars to roughly same scale
rich <- dat$rich/100
abiotic <- dat$abiotic/100
hetero <- dat$hetero
age <- dat$age/100
firesev <- dat$firesev/10
distance <- dat$distance/100


In [34]:
### Create Transformed Dataset
# overwrite file with recoded data
t.dat <- data.frame(rich,abiotic ,hetero ,age, firesev,distance)
summary(t.dat)

      rich           abiotic           hetero            age        
 Min.   :0.1500   Min.   :0.3259   Min.   :0.3842   Min.   :0.0300  
 1st Qu.:0.3700   1st Qu.:0.4381   1st Qu.:0.6246   1st Qu.:0.1500  
 Median :0.5000   Median :0.4804   Median :0.6843   Median :0.2500  
 Mean   :0.4923   Mean   :0.4924   Mean   :0.6833   Mean   :0.2557  
 3rd Qu.:0.6200   3rd Qu.:0.5490   3rd Qu.:0.7684   3rd Qu.:0.3500  
 Max.   :0.8500   Max.   :0.7046   Max.   :0.8779   Max.   :0.6000  
    firesev          distance     
 Min.   :0.1200   Min.   :0.3704  
 1st Qu.:0.3700   1st Qu.:0.3946  
 Median :0.4300   Median :0.5177  
 Mean   :0.4565   Mean   :0.4923  
 3rd Qu.:0.5550   3rd Qu.:0.5840  
 Max.   :0.9200   Max.   :0.6072  

In [35]:
# let's rebuild the model

mod.1 <- 'rich ~ abiotic + hetero + distance + firesev + 0*age
    abiotic ~ distance
    hetero ~ distance
    age ~ distance
    firesev ~ age'

In [36]:
mod.1.fit <- sem(mod.1, data=t.dat)

In [37]:
summary(mod.1.fit)

lavaan (0.6-1) converged normally after  41 iterations

  Number of observations                            90

  Estimator                                         ML
  Model Fit Test Statistic                       6.900
  Degrees of freedom                                 7
  P-value (Chi-square)                           0.439

Parameter Estimates:

  Information                                 Expected
  Information saturated (h1) model          Structured
  Standard Errors                             Standard

Regressions:
                   Estimate  Std.Err  z-value  P(>|z|)
  rich ~                                              
    abiotic           0.475    0.163    2.909    0.004
    hetero            0.352    0.103    3.410    0.001
    distance          0.550    0.150    3.663    0.000
    firesev          -0.195    0.068   -2.874    0.004
    age               0.000                           
  abiotic ~                                           
    distance          0.40

In [ ]:
## The p-value indicates a good fit

### Let's try model 2 (includes a direct effect)

In [38]:
# specify “mod.2”
mod.2 <- 'rich ~ abiotic + hetero + distance + firesev + age
    abiotic ~ distance
    hetero ~ distance
    age ~ distance
    firesev ~ age'

In [40]:
mod.2.fit <- sem(mod.2, data=t.dat)

In [41]:
summary(mod.2.fit)

lavaan (0.6-1) converged normally after  42 iterations

  Number of observations                            90

  Estimator                                         ML
  Model Fit Test Statistic                       6.095
  Degrees of freedom                                 6
  P-value (Chi-square)                           0.413

Parameter Estimates:

  Information                                 Expected
  Information saturated (h1) model          Structured
  Standard Errors                             Standard

Regressions:
                   Estimate  Std.Err  z-value  P(>|z|)
  rich ~                                              
    abiotic           0.481    0.162    2.959    0.003
    hetero            0.350    0.103    3.401    0.001
    distance          0.528    0.153    3.449    0.001
    firesev          -0.167    0.075   -2.215    0.027
    age              -0.091    0.102   -0.886    0.376
  abiotic ~                                           
    distance          0.40

#### Overall the model seems to fit well (p-value=0.413>0)
#### We see that the direct effect of age on richness is neither relevant (estimate=-0.091) nor statistically significant (0.376)

In [43]:
anova(mod.1.fit, mod.2.fit)

,Df,AIC,BIC,Chisq,Chisq diff,Df diff,Pr(>Chisq)
mod.2.fit,6,-715.3471,-680.3498,6.094552,NA,NA,NA
mod.1.fit,7,-716.5419,-684.0444,6.899752,0.8052001,1,0.3695432


### The previous result shows that the model with direct path from age to rich (mod.2.fit) is not significantly better than the first one (no direct mediation)

### We select the first model as the one close enough to the data

In [49]:
standardizedSolution(mod.2.fit, type = "std.all")

lhs,op,rhs,est.std,se,z,pvalue,ci.lower,ci.upper
rich,~,abiotic,0.25030004,0.08356737,2.9951886,2.742752e-03,0.0865110,0.41408908
rich,~,hetero,0.27236810,0.07880941,3.4560352,5.481837e-04,0.1179045,0.42683171
rich,~,distance,0.31606436,0.08802804,3.5904965,3.300487e-04,0.1435326,0.48859615
rich,~,firesev,-0.18678847,0.08394089,-2.2252381,2.606526e-02,-0.3513096,-0.02226735
rich,~,age,-0.07713315,0.08701852,-0.8863993,3.754024e-01,-0.2476863,0.09342002
abiotic,~,distance,0.45972328,0.07861649,5.8476697,4.985072e-09,0.3056378,0.61380877
hetero,~,distance,0.34598753,0.08997120,3.8455365,1.202890e-04,0.1696472,0.52232784
age,~,distance,-0.27814383,0.09535483,-2.9169350,3.534894e-03,-0.4650358,-0.09125180
firesev,~,age,0.45386542,0.08366979,5.4244837,5.812228e-08,0.2898756,0.61785520
rich,~~,rich,0.50803719,0.06964202,7.2949807,2.986500e-13,0.3715413,0.64453304
